# Project

In [159]:
using CSV, DataFrames, JuMP, Gurobi, Plots, LinearAlgebra, Dates

In [28]:
fires = CSV.read("data/wildfire_suppression.csv", DataFrame, header=true);
first(fires, 5)

Row,report_id,fire_id,area,date,longitude,latitude,from_date,crews_sent,total_crews_sent,max_crews_sent
,Int64,Int64,Float64,String31,Float64,Float64,String31,Float64,Float64,Float64
1,2714023,2714022,150.0,2015-05-06 10:15:00,-71.2519,43.7811,2015-05-05 09:30:00,45.0,123.0,45.0
2,2714037,2714022,275.0,2015-05-08 00:30:00,-71.2519,43.7811,2015-05-07 11:00:00,45.0,123.0,45.0
3,2714050,2714022,275.0,2015-05-09 00:30:00,-71.2519,43.7811,2015-05-08 11:00:00,33.0,123.0,45.0
4,2714066,2714022,275.0,2015-05-10 13:00:00,-71.2519,43.7811,2015-05-09 15:00:00,0.0,123.0,45.0
5,2714082,2714081,205.0,2015-05-07 07:15:00,-67.0125,44.7917,2015-05-06 20:30:00,21.0,37.0,21.0


In [29]:
function parse_date(date_string)
    try
        return DateTime(date_string, "yyyy-mm-dd HH:MM:SS.sss")
    catch
        return DateTime(date_string, "yyyy-mm-dd HH:MM:SS")
    end
end
fires.date = [parse_date(date_string) for date_string in fires.date]
fires_df = filter(row -> Dates.format(row.date, "yyyy-mm-dd") == "2017-08-01", fires);

### Preprocessing

In [30]:
Fires = Matrix(fires_df);
n_fires = 1:nrow(fires_df);
Surfaces = fires_df[:, :area];


### Distance feature

In [31]:
# Calculate Euclidian distance
function euclidean_distance(lat1, lon1, lat2, lon2)
    return sqrt((lat2 - lat1)^2 + (lon2 - lon1)^2)
end

# Calculate the Haversine distance
function haversine_distance(lat1, lon1, lat2, lon2)
    R = 6371.0 # Earth radius in kilometers
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    lat1 = deg2rad(lat1)
    lat2 = deg2rad(lat2)

    a = sin(dLat/2)^2 + cos(lat1) * cos(lat2) * sin(dLon/2)^2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c
end

# Returns distance matrix of distances between fires
function distance_matrix(df, euclidian=1)
    n = nrow(df)
    dist_matrix = zeros(n, n)
    for i in 1:n
        for j in (i+1):n
            if euclidian == 1
                dist = euclidean_distance(df[i, :latitude], df[i, :longitude], df[j, :latitude], df[j, :longitude])
            else
                dist = haversine_distance(df[i, :latitude], df[i, :longitude], df[j, :latitude], df[j, :longitude])
            end
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
        end
    end
    return dist_matrix
end;

Distances = distance_matrix(fires_df)

61×61 Matrix{Float64}:
  0.0         0.881321  10.6814   10.6814   …   1.09791   10.9593     4.38287
  0.881321    0.0       10.8284   10.8284       0.674199  10.6799     4.2979
 10.6814     10.8284     0.0       0.0         11.486      5.23505    6.6748
 10.6814     10.8284     0.0       0.0         11.486      5.23505    6.6748
  1.27371     0.404295  10.8014   10.8014       0.887146  10.4648     4.20086
  9.2609      8.93471    5.59216   5.59216  …   9.57861    1.81223    5.08603
 10.7762     10.4921     5.24959   5.24959     11.1438     0.195244   6.49889
  1.36365     1.61534   12.037    12.037        1.09556   12.2396     5.72394
  2.04487     2.02291    8.80576   8.80576      2.68536    8.93202    2.34194
  4.542       3.67103   11.5617   11.5617       3.85727    9.64937    5.17154
  0.0858781   0.966676  10.6808   10.6808   …   1.16072   10.999      4.41006
  5.60245     5.71621    5.11224   5.11224      6.37477    6.24567    1.7328
  0.146176    0.988766  10.7731   10.7731    

### First model

In [33]:
n_fighters = Int(sum(fires_on_august_1_2017[:,:crews_sent]));
demand = [];
n_fires = [];
days = ["2017-08-01", "2017-08-02", "2017-08-03"];

for day in days
    fires_on_day_i = filter(row -> Dates.format(row.date, "yyyy-mm-dd") == day, fires)
    
    push!(n_fires, nrow(unique(fires_on_day_i, :fire_id)));
    
    # Group by fire_id and calculate sum of max_crews_sent for each group
    grouped_fires = groupby(fires_on_day_i, :fire_id)
    sum_max_crews_sent = [sum(group[!,:max_crews_sent]) for group in grouped_fires]

    # Push the array of sums to the demand array
    push!(demand, sum_max_crews_sent)
end

In [34]:
grouped_fires = groupby(fires_on_august_1_2017, :fire_id)
sum_crews_sent = [sum(group[!,:crews_sent]) for group in grouped_fires];

assignements = zeros(Int(n_fighters), n_fires[1]);
k, start = 1, 1
for f in sum_crews_sent
    num_ones = Int(f)
    assignements[start:num_ones, k] .= 1
    start = Int(f)+1
    k += 1
end  

In [35]:
n_fighters

17782

In [158]:
n_fires

3-element Vector{Any}:
 51
 53
 49

In [61]:
n_groups=1000

1000

In [81]:
fire_demand = sum(demand[1][k] for k=1:n_fires[1])/n_fires[1]

795.4901960784314

In [ ]:
model = Model(Gurobi.Optimizer)

In [72]:

model = Model(Gurobi.Optimizer)

# Redefine the variables with groups instead of individual fighters
@variable(model, X1[g=1:n_groups, k=1:n_fires[1], t=1:3], binary = true)
@variable(model, Y1[g=1:n_groups, k1=1:n_fires[1], k2=1:n_fires[1], t=2:3], binary = true)

# New variables for positive parts of costs
@variable(model, pos_cost[k=1:n_fires[1], t=1:3] >= 0)

# Constraints for new variables
@constraint(model, pos_cost_con[k=1:n_fires[1], t=1:3],
            pos_cost[k, t] >= demand[1][k] - sum(sum(X1[g,k,tprime] for g=1:n_groups for tprime=1:t))

# Adjusted objective function
@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires[1], t=1:3) + 
                        0.0000001 * sum(Y1[g,k1,k2,t] * Distances[k1,k2] for g=1:n_groups, k1=1:n_fires[1], k2=1:n_fires[1], t=2:3))

# Adjusted constraints for groups
@constraint(model, constraint_fighters1[t=1:3, g=1:n_groups],
            sum(X1[g,k,t] for k=1:n_fires[1]) == 1)
@constraint(model, constraint_Y1[t=2:3, k1=1:n_fires[1], k2=1:n_fires[1], g=1:n_groups],
            X1[g,k1,t-1] + X1[g,k2,t] - 1 <= Y1[g,k1,k2,t])

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 520653 rows, 535653 columns and 1606653 nonzeros
Model fingerprint: 0x0f72418a
Variable types: 153 continuous, 535500 integer (535500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-09, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 121122.00017
Presolve removed 13900 rows and 13900 columns
Presolve time: 3.30s
Presolved: 506753 rows, 521753 columns, 1546253 nonzeros
Variable types: 0 continuous, 521753 integer (521703 binary)
Found heuristic solution: objective 121110.00017
Determinis

In [73]:
X = value.(X1)
Y = value.(Y1);

In [77]:
indices = [argmax(X[4, :, i]) for i in 1:3]

3-element Vector{Int64}:
 21
 42
  1

In [84]:
n_groups = 100
model = Model(Gurobi.Optimizer)

# Redefine the variables with groups instead of individual fighters
@variable(model, X1[g=1:n_groups, k=1:n_fires[1], t=1:3], binary = true)
@variable(model, Y1[g=1:n_groups, k1=1:n_fires[1], k2=1:n_fires[1], t=2:3], binary = true)

# New variables for positive parts of costs
@variable(model, pos_cost[k=1:n_fires[1], t=1:3] >= 0)

# Constraints for new variables
@constraint(model, pos_cost_con[k=1:n_fires[1], t=1:3],
            pos_cost[k, t] >= fire_demand - sum(sum(X1[g,k,tprime] for g=1:n_groups for tprime=1:t)))

# Adjusted objective function
@objective(model, Min, sum(pos_cost[k, t] for k=1:n_fires[1], t=1:3) + 
                        0.0000001 * sum(Y1[g,k1,k2,t] * Distances[k1,k2] for g=1:n_groups, k1=1:n_fires[1], k2=1:n_fires[1], t=2:3))

# Adjusted constraints for groups
@constraint(model, constraint_fighters1[t=1:3, g=1:n_groups],
            sum(X1[g,k,t] for k=1:n_fires[1]) == 1)
@constraint(model, constraint_Y1[t=2:3, k1=1:n_fires[1], k2=1:n_fires[1], g=1:n_groups],
            X1[g,k1,t-1] + X1[g,k2,t] - 1 <= Y1[g,k1,k2,t])

set_optimizer_attribute(model, "TimeLimit", 60.0)
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 520653 rows, 535653 columns and 1606653 nonzeros
Model fingerprint: 0x0a19ea68
Variable types: 153 continuous, 535500 integer (535500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-09, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 121110.00016
Presolve removed 515586 rows and 530436 columns (presolve time = 6s) ...
Presolve removed 515586 rows and 530436 columns
Presolve time: 6.12s
Presolved: 5067 rows, 5217 columns, 15345 nonzeros
Found heuristic solution: objective 121110.00000
Va

In [100]:
X1 = value.(X1);
Y1 = value.(Y1);

In [ ]:
sum(pos_cost[k, t] for k=1:n_fires[1], t=1:3) + 
                        0.0000001 * sum(Y1[g,k1,k2,t] * Distances[k1,k2] for g=1:n_groups, k1=1:n_fires[1], k2=1:n_fires[1], t=2:3))

In [155]:
function compute_objective(X1, Y1, Distances, fire_demand, n_fires)
    # Calculate pos_cost
    pos_cost = zeros(n_fires[1], 3)
    for k in 1:n_fires[1]
        for t in 1:3
            sum_x1 = sum(sum(X1[g, k, tprime] for g in 1:size(X1, 1) for tprime in 1:t))
            pos_cost[k, t] = max(fire_demand - sum_x1, 0)
        end
    end

    # Compute the two parts of the objective function
    part1 = sum(pos_cost)
    part2 = 0.0000001 * sum(Y1[g, k1, k2, t] * Distances[k1, k2] for g in 1:size(Y1, 1), k1 in 1:n_fires[1], k2 in 1:n_fires[1], t in 2:3)

    return part1 + part2
end


compute_objective (generic function with 1 method)

In [156]:
n_fires[1]

51

In [157]:
compute_objective(X1, Y1, Distances, fire_demand, n_fires)

121110.00000157651

In [91]:
using Pkg
Pkg.add("PlotlyJS")


    Updating registry at `C:\Users\benja\.julia\registries\General.toml`
   Resolving package versions...
   Installed Hiccup ──────────────── v0.2.2
   Installed AssetRegistry ───────── v0.1.0
   Installed Pidfile ─────────────── v1.3.0
   Installed PlotlyBase ──────────── v0.8.19
   Installed WebSockets ──────────── v1.6.0
   Installed Blink ───────────────── v0.12.8
   Installed Kaleido_jll ─────────── v0.2.1+0
   Installed JSExpr ──────────────── v0.5.4
   Installed Mux ─────────────────── v1.0.1
   Installed Mustache ────────────── v1.0.19
   Installed FunctionalCollections ─ v0.5.0
   Installed PlotlyJS ────────────── v0.18.11
   Installed WebIO ───────────────── v0.8.21
    Updating `C:\Users\benja\.julia\environments\v1.9\Project.toml`
  [f0f68f2c] + PlotlyJS v0.18.11
    Updating `C:\Users\benja\.julia\environments\v1.9\Manifest.toml`
  [bf4720bc] + AssetRegistry v0.1.0
  [ad839575] + Blink v0.12.8
  [de31a74c] + FunctionalCollections v0.5.0
  [9fb69e20] + Hiccup v0.2.2
  [97c

In [134]:
using DataFrames, PlotlyJS, CSV

traces = []
for (id, lat, lon) in eachrow(fires_on_august_1_2017[!, [:fire_id, :latitude, :longitude]])
   push!(traces, scattergeo(lon=[lon], lat=[lat], mode="markers+text", text=["Fire ID: $id"], textposition="bottom center"))
end

layout = Layout(
    title="Fire Crew Unit Movements Over Time",
    geo=attr(
        scope="usa",
        projection_type="albers usa",
        showland=true,
        landcolor="rgb(250, 250, 250)",
        subunitcolor="rgb(217, 217, 217)",
        countrycolor="rgb(217, 217, 217)",
        countrywidth=0.5,
        subunitwidth=0.5
    )
)

PlotlyJS.plot(traces[1], layout)


data: [
  "scattergeo with fields lat, lon, mode, text, textposition, and type"
]

layout: "layout with fields geo, margin, template, and title"

61-element Vector{Any}:
 GenericTrace{Dict{Symbol, Any}}(Dict{Symbol, Any}(:mode => "markers+text", :textposition => "bottom center", :type => "scattergeo", :lat => [7133019], :text => ["Fire ID: 7133137"], :lon => [3621.0]))
 GenericTrace{Dict{Symbol, Any}}(Dict{Symbol, Any}(:mode => "markers+text", :textposition => "bottom center", :type => "scattergeo", :lat => [7165456], :text => ["Fire ID: 7166347"], :lon => [5724.0]))
 GenericTrace{Dict{Symbol, Any}}(Dict{Symbol, Any}(:mode => "markers+text", :textposition => "bottom center", :type => "scattergeo", :lat => [7170139], :text => ["Fire ID: 7170884"], :lon => [81826.0]))
 GenericTrace{Dict{Symbol, Any}}(Dict{Symbol, Any}(:mode => "markers+text", :textposition => "bottom center", :type => "scattergeo", :lat => [7170139], :text => ["Fire ID: 7170401"], :lon => [81826.0]))
 GenericTrace{Dict{Symbol, Any}}(Dict{Symbol, Any}(:mode => "markers+text", :textposition => "bottom center", :type => "scattergeo", :lat => [7173747], :text => ["Fir

In [112]:
using Pkg
Pkg.add("PlotlyJS")


   Resolving package versions...
  No Changes to `C:\Users\benja\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\benja\.julia\environments\v1.9\Manifest.toml`


In [102]:
using Conda
Conda.pip_interop(true)
Conda.pip("install", "webio_jupyter_extension")

[ Info: Running `conda config --set pip_interop_enabled true --file 'C:\Users\benja\.julia\conda\3\x86_64\condarc-julia.yml'` in root environment
[ Info: Running `pip install webio_jupyter_extension` in root environment


     ------------------------------------- 219.9/219.9 kB 1.2 MB/s eta 0:00:00


In [135]:
using PlotlyJS

# Example data (replace with your actual data)
fire_locations = Dict(
    1 => (lat = 34.0522, lon = -118.2437),  # Fire 1 coordinates
    2 => (lat = 36.7783, lon = -119.4179)   # Fire 2 coordinates
)

# Replace with your actual tensor data
X = rand(0:1, 5, 3, length(fire_locations))  # 5 units, 3 days, N fires

# Create traces for each unit and day
traces = []
for i in 1:size(X, 1)  # for each fire crew unit
    for t in 1:size(X, 2)  # for each day
        last_location = nothing
        for k in 1:size(X, 3)  # for each fire
            if X[i, t, k] == 1
                location = fire_locations[k]
                push!(traces, scattergeo(lon=[location.lon], lat=[location.lat], mode="markers+text", text=["Unit $i"], textposition="bottom center"))

                if last_location !== nothing
                    # Draw line for movement
                    push!(traces, scattergeo(lon=[last_location.lon, location.lon], lat=[last_location.lat, location.lat], mode="lines", line=attr(width=2, color="blue")))
                end
                last_location = location
            end
        end
    end
end

# Creating the layout
layout = Layout(
    title="Macro Movement of Fire Crew Units",
    geo=attr(
        scope="usa",
        projection_type="albers usa",
        showland=true,
        landcolor="rgb(250, 250, 250)",
        subunitcolor="rgb(217, 217, 217)",
        countrycolor="rgb(217, 217, 217)",
        countrywidth=0.5,
        subunitwidth=0.5
    )
)

# Plot the figure
fig = plot(traces, layout)
PlotlyJS.show(fig)


LoadError: UndefVarError: `plot` not defined